In [3]:
!pip install torch torchvision transformers datasets pillow

  Using cached torch-2.6.0-cp312-none-macosx_11_0_arm64.whl.metadata (28 kB)
  Using cached torchvision-0.21.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (6.1 kB)
  Using cached transformers-4.49.0-py3-none-any.whl.metadata (44 kB)
  Using cached networkx-3.4.2-py3-none-any.whl.metadata (6.3 kB)
  Using cached fsspec-2025.3.0-py3-none-any.whl.metadata (11 kB)
  Using cached sympy-1.13.1-py3-none-any.whl.metadata (12 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
  Using cached huggingface_hub-0.29.3-py3-none-any.whl.metadata (13 kB)
  Using cached safetensors-0.5.3-cp38-abi3-macosx_11_0_arm64.whl.metadata (3.8 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached xxhash-3.5.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (12 kB)
  Using cached multiprocess-0.70.16-py312-none-any.whl.metadata (7.2 kB)
  Using cached fsspec-2024.12.0-py3-none-any.whl.metadata (11 kB)
  Using cached aios

In [10]:
conda install -n base -c conda-forge jupyterlab_widgets


Retrieving notices: done
Channels:
 - conda-forge
 - defaults
Platform: osx-arm64
Solving environment: done

## Package Plan ##

  environment location: /opt/homebrew/anaconda3

  added / updated specs:
    - jupyterlab_widgets


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2025.1.31          |     pyhd8ed1ab_0         159 KB  conda-forge
    conda-25.1.1               |  py312h81bd7bf_1         1.1 MB  conda-forge
    ipywidgets-8.1.5           |     pyhd8ed1ab_1         111 KB  conda-forge
    jupyterlab_widgets-3.0.13  |     pyhd8ed1ab_1         182 KB  conda-forge
    libexpat-2.6.3             |       hf9b8971_0          62 KB  conda-forge
    libsqlite-3.46.0           |       hfb93653_0         811 KB  conda-forge
    libzlib-1.2.13             |       hfb2fe0b_6          46 KB  conda-forge
    openssl-3.4.1              |       h81ee809_0         2.8 MB  conda-forge
  

In [12]:
conda install -n pyenv -c conda-forge ipywidgets


EnvironmentLocationNotFound: Not a conda environment: /opt/homebrew/anaconda3/envs/pyenv


Note: you may need to restart the kernel to use updated packages.


### Gerekli Kutuphaneleri Yukleme

In [33]:
import os
import json
import torch
import pandas as pd
import requests
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from transformers import DistilBertTokenizer, DistilBertModel
from tqdm import tqdm


In [35]:
os.makedirs("coco",exist_ok=True)

### İndirme fonksiyonu  
### COCO veri setini indirme goruntu ve aciklama(goruntu+metin) dan olusan  bir dataset

In [38]:

def download_file(url, save_path):
    response = requests.get(url, stream=True)
    total_size = int(response.headers.get("content-length", 0))
    block_size = 1024  # 1 KB
    t = tqdm(total=total_size, unit="iB", unit_scale=True)

    with open(save_path, "wb") as file:
        for data in response.iter_content(block_size):
            t.update(len(data))
            file.write(data)
    t.close()

# COCO veri seti URL'leri
coco_urls = {
    "train2017.zip": "http://images.cocodataset.org/zips/train2017.zip",
    "annotations_trainval2017.zip": "http://images.cocodataset.org/annotations/annotations_trainval2017.zip"
}

In [41]:
# Dosyaları indir
for filename, url in coco_urls.items():
    save_path = os.path.join("coco", filename)
    if not os.path.exists(save_path):
        print(f"İndiriliyor: {filename}")
        download_file(url, save_path)
    else:
        print(f"Zaten mevcut: {filename}")

# Dosyaları aç
import zipfile

for filename in coco_urls.keys():
    zip_path = os.path.join("coco", filename)
    extract_path = "coco"
    with zipfile.ZipFile(zip_path, "r") as zip_ref:
        zip_ref.extractall(extract_path)
        print(f"Açıldı: {zip_path}")

İndiriliyor: train2017.zip


100%|███████████████████████████████████| 19.3G/19.3G [1:07:00<00:00, 4.81MiB/s]


İndiriliyor: annotations_trainval2017.zip


100%|███████████████████████████████████████| 253M/253M [00:51<00:00, 4.89MiB/s]


Açıldı: coco/train2017.zip
Açıldı: coco/annotations_trainval2017.zip


### COCO dataset in Anotasyonlarini Okuma

In [46]:
!pip install pycocotools


In [48]:
from pycocotools.coco import COCO
import os

# COCO Anotasyon dosyasının yolu
annotation_path = "coco/annotations/captions_train2017.json"

# COCO API'yi başlat
coco = COCO(annotation_path)

# Tüm görüntülerin ID'lerini al
image_ids = list(coco.imgs.keys())

# İlk birkaç görüntüye bakalım
print("Toplam Görüntü Sayısı:", len(image_ids))
print("İlk Görüntü ID’leri:", image_ids[:5])


loading annotations into memory...
Done (t=0.64s)
creating index...
index created!
Toplam Görüntü Sayısı: 118287
İlk Görüntü ID’leri: [391895, 522418, 184613, 318219, 554625]


In [50]:
class COCODataset(Dataset):
    def __init__(self, root_dir, annotation_path, transform=None):
        self.root_dir = root_dir
        self.coco = COCO(annotation_path)
        self.image_ids = list(self.coco.imgs.keys())
        self.transform = transform

    def __len__(self):
        return len(self.image_ids)

    def __getitem__(self, index):
        # Rastgele bir görüntü ID'si seç
        img_id = self.image_ids[index]
        
        # Görüntü yolunu al
        img_info = self.coco.loadImgs(img_id)[0]
        img_path = os.path.join(self.root_dir, img_info["file_name"])
        
        # Görüntüyü aç
        image = Image.open(img_path).convert("RGB")
        
        # Dönüştürme işlemleri
        if self.transform:
            image = self.transform(image)
        
        # Görüntüye karşılık gelen açıklamaları al
        annotation_ids = self.coco.getAnnIds(imgIds=img_id)
        annotations = self.coco.loadAnns(annotation_ids)
        
        # Rastgele bir açıklama seç
        caption = random.choice(annotations)["caption"]

        return image, caption


In [52]:
# Görüntü dönüştürme işlemleri
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Görüntüyü 224x224 boyutuna getir
    transforms.ToTensor(),  # Tensor formatına çevir
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalizasyon
])


In [56]:
from torch.utils.data import DataLoader
import random

# Veri setini başlat
coco_dataset = COCODataset(
    root_dir="coco/train2017",
    annotation_path="coco/annotations/captions_train2017.json",
    transform=transform
)

# DataLoader oluştur
data_loader = DataLoader(coco_dataset, batch_size=8, shuffle=True)

# Örnek bir batch alalım
images, captions = next(iter(data_loader))

print("Batch Boyutu:", images.shape)  # (8, 3, 224, 224)
print("Örnek Açıklamalar:", captions)


loading annotations into memory...
Done (t=0.66s)
creating index...
index created!
Batch Boyutu: torch.Size([8, 3, 224, 224])
Örnek Açıklamalar: ('A woman on snow shoes with poles standing on the ground.', 'Teddy bears dressed in sweaters and scarfs sit on a bench holding a book. ', 'pasta salad and a burger on a plate', 'THIS IS A PHOTO OF A BEAUTIFUL HORSE AT THE FENCE', 'Polar bear swimming through river looking at camera.', 'A group of children playing baseball out side.', 'Group of people sitting at a dinner table with lots of food. ', 'A room with a bed, tables, a coach and a patio outside.')
